#**MENYIAPKAN LIBRARY**

In [ ]:
# Download some library
!pip install nltk
import nltk
nltk.download("popular")

In [1]:
!pip install Sastrawi
!pip install sklearn

     |████████████████████████████████| 215kB 11.1MB/s 


In [2]:
#from nltk.tokenize import word_tokenize
#from nltk.tokenize import RegexpTokenizer
#from nltk.corpus import stopwords
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [4]:
from google.colab import files, output

import random as rd
import pandas as pd
import numpy as np
import requests
import math
import time   
import io
import re

#**MENGAMBIL DATASET**

##**DATASET ASLI**

In [24]:
#Dataset Asli
link = 'https://drive.google.com/open?id=1d-KMfYnE5dWPRchKnMe6ASGPk3ntasoi'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1d-KMfYnE5dWPRchKnMe6ASGPk3ntasoi


In [25]:
#================================ DATA FRAME =================================
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Dataset.csv')  
dfRAW = pd.read_csv('Dataset.csv', encoding = "ISO-8859-1")

cols = [0,1,2,3]
dfDataset = dfRAW[dfRAW.columns[cols]]

#dfDataset.sort_values(by=["Kelas Berita"], inplace=True)
#dfDataset = dfDataset.reset_index(drop=True)

dfDataset

,Unnamed: 0,Judul Berita,Isi Berita,Kelas Berita
0,1,"Usai Gelar Doa Bersama, Prabowo Target Raih 63...","Jakarta, CNN Indonesia -- Calon Presiden Nomor...",Negatif
1,2,"Kubu Prabowo Klaim Menang Versi Exit Poll, PDI...","Liputan6.com, Jakarta - Sekjen PDI Perjuangan ...",Negatif
2,3,"Yenny Wahid: Jokowi Bukan Orang 'Ge-eran', Tak...",Putri Presiden RI ke-4 Abudrrahman Wahid (Gus ...,Negatif
3,4,"Quick Count LSI Denny JA 98,85%: Jokowi 55,71%...",Berbagai lembaga survei menggelar hitung cepat...,Positif
4,5,Prabowo - Sandi unggul sementara di Kaitetu,Ambon (ANTARA) - Pasangan calon presiden dan w...,Positif
...,...,...,...,...
95,96,Grace Natalie Akui 'Kekalahan' PSI,Hasil quick count lembaga survei menunjukkan h...,Negatif
96,97,Prabowo Menang Telak di Wilayah Kediaman Ustaz...,"Jakarta, CNN Indonesia -- Pasangan calon presi...",Negatif
97,98,TKN Jokowi: Survei Internal BPN Sifatnya 'Asal...,Wakil Ketua Tim Kampanye Nasional (TKN) Joko W...,Positif
98,99,Malaysia ingin adopsi masa tenang pemilu Indon...,Jakarta (ANTARA) - Pemantau pemilu asal Malays...,Positif


##**DATASET KECIL**

In [5]:
uploaded = files.upload()

Saving Dataset Kecil.xlsx to Dataset Kecil.xlsx


In [6]:
dfRAW = pd.read_excel(io.BytesIO(uploaded['Dataset Kecil.xlsx']))
#dfRAW = pd.read_csv(io.BytesIO(uploaded['Dataset.csv']), encoding = "ISO-8859-1")

dfDataset = dfRAW

#dfDataset.sort_values(by=["Kelas Berita"], inplace=True)
#dfDataset = dfDataset.reset_index(drop=True)

dfDataset

,Unnamed: 0,Judul Berita,Isi Berita,Kelas Berita
0,1,Prabowo Muak dengan Elite di Jakarta: Mereka A...,"Liputan6.com, Yogyakarta - Calon Presiden Prab...",Negatif
1,2,Jokowi: Pembuat Kebohongan Itu Tidak Mikir,"Liputan6.com, Karawang - Capres nomor urut 01 ...",Negatif
2,3,Jokowi: Saya Tanya PUBG Dijawab Pak Prabowo Pe...,Capres petahana Joko Widodo (Jokowi) menyinggu...,Negatif
3,4,Pemantau Pemilu Asing: Indonesia Seharusnya Ba...,"Jakarta, CNN Indonesia -- Pemantau pemilu yang...",Positif
4,5,Tips Jokowi Jaga Kebugaran Tubuh Selama Kampanye,MAMUJU - Calon presiden pejawat Joko Widodo (J...,Positif
5,6,BPN Sebut KPU Pulihkan 944 Ribu DPT Bermasalah,"Jakarta, CNN Indonesia -- Badan Pemenangan Nas...",Positif
6,7,"Debat Kelima, TKN: Prabowo Tak Nyambung",TKN Jokowi-Ma'ruf mengkritik capres 02 Prabowo...,Negatif
7,8,Pesan Politik SBY Jelang Pemungutan Suara Pemi...,"Jakarta, CNN Indonesia -- Presiden keenam Repu...",Positif


#**MEMISAHKAN DATA LATIH & DATA UJI**

In [7]:
def getDataLatih(dfPositif, dfNegatif, drainData):
  dfLatihPositif = dfPositif.head(int(drainData/2))
  dfLatihNegatif = dfNegatif.head(int(drainData/2))
  
  dfDataLatih = dfLatihPositif.append(dfLatihNegatif)
  dfDataLatih = dfDataLatih.iloc[dfDataLatih["Unnamed: 0"].argsort()]

  return dfDataLatih

In [8]:
def getDataUji(dfDataset, dfDataLatih):
  key_diff = set(dfDataset["Unnamed: 0"]).difference(dfDataLatih["Unnamed: 0"])
  where_diff = dfDataset["Unnamed: 0"].isin(key_diff)

  dfDataUji = dfDataset[where_diff]
  dfDataUji = dfDataUji.iloc[dfDataUji["Unnamed: 0"].argsort()]

  return dfDataUji

#**PREPROCESSING TEXT**

##**TOKENIZING**

In [31]:
# MEMBUAT FUNCTION TOKENIZE UNTUK MEMISAHKAN SETIAP KATA PADA DATA 
def Tokenize(Value):
  punc_Num='¢Â€Ã!"#$%&\'()*+,:;<=>?@[\\]^_`{|}~0123456789'
  for ch in punc_Num:                       #--> PERULANGAN UNTUK MENGECEK CHARACTER BERDASARKAN "punc_Num"
    Value = Value.replace(ch, "")           #--> MENGHAPUS CHARACTER 
  Value = Value.replace(".", " ")           #--> MENGGANTIKAN CHARACTER "." MENJADI WHITESPACE " "
  Value = Value.replace("-", " ")           #--> MENGGANTIKAN CHARACTER "-" MENJADI WHITESPACE " "
  Value = Value.replace("/", " ")           #--> MENGGANTIKAN CHARACTER "/" MENJADI WHITESPACE " "
  Value = Value.replace("‚", " ")           #--> MENGGANTIKAN CHARACTER "," MENJADI WHITESPACE " "
  Value = Value.replace("\xa0", " ")        #--> MENGGANTIKAN CHARACTER "\xa0" MENJADI WHITESPACE " "  
  Value = Value.replace("\x80\x9c", " ")  #--> MENGGANTIKAN CHARACTER "\x80\x9c" MENJADI WHITESPACE " "
  Value = Value.replace("\x80", " ")      #--> MENGGANTIKAN CHARACTER "\x80" MENJADI WHITESPACE " "
  Value = Value.replace("\x93", " ")      #--> MENGGANTIKAN CHARACTER "\x93" MENJADI WHITESPACE " "
  Value = Value.replace("\x94", " ")      #--> MENGGANTIKAN CHARACTER "\x94" MENJADI WHITESPACE " "
  Value = Value.replace("\x99", " ")      #--> MENGGANTIKAN CHARACTER "\x99" MENJADI WHITESPACE " "

  Token = Value.split()   #--> MEMISAHKAN SETIAP KATA
  
  return Token

In [10]:
def Tokenizing(dfDataLatih, dfDataUji):
  preparation = dfDataLatih
  preparation = preparation.append(dfDataUji)
  preparation = preparation.reset_index(drop=True)

  tokenized = preparation["Raw Text"]

  preprocessing = pd.DataFrame(tokenized, columns = ['tokenized']) 
  preprocessing['tokenized'] = tokenized
  preprocessing['tokenized'] = preprocessing['tokenized'].apply(lambda x: Tokenize(x))

  print("  Tokenizing Done. . .")
  return preprocessing

##**FILTERING**

In [11]:
def Filtering(preprocessing):
  #stopList = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z",]
  stopList = []
  listBerita = ["cnn", "cnnindonesia", "cnnindonesiacom", "detik", "detikcom", "liputan", "liputancom", "inipasti", "inipasticom", "antaranews", "antaranewscom", "com"]

  url = "http://static.hikaruyuuki.com/wp-content/uploads/stopword_list_tala.txt"
  headers = {'Accept-Encoding': 'identity'}
  r = requests.get(url, headers=headers); ind = 0
  for line in r.text.splitlines():
      #print(line)
      stopList.append(line)
      ind += 1
      if len(r.text.splitlines()) == ind:
        for berita in listBerita:
          stopList.append(berita)

  preprocessing['filtering'] = preprocessing['tokenized'].apply(lambda x: [item for item in x if item.lower() not in stopList])

  print("  Filtering Done. . .")
  return preprocessing

##**STEMMING**

In [12]:
def Stemming(preprocessing):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  preprocessing['stemming'] = preprocessing['filtering'].apply(lambda x: [stemmer.stem(y) for y in x])
  #preprocessing['stemming'] = preprocessing['stemming'].apply(lambda x: [item for item in x if item not in stopList])

  print("  Stemming Done. . .")
  return preprocessing

##**PEMBOBOTAN KATA**

In [13]:
def TermUniques(prepDataLatih):
  teksIndeks = prepDataLatih['stemming'].to_numpy()
  clearArray = np.concatenate(teksIndeks, axis=0 )

  uniques = np.unique(clearArray)
  uniques = uniques.tolist()
  if '' in uniques:
    uniques.remove('')

  return uniques

In [14]:
def TermWeighting(prepDataLatih, uniques):
  prepDataLatih = dfPrepDataLatih
  bobotData = pd.DataFrame(index=[teks for teks in uniques], columns=[("D" + str(i+1)) for i in range(len(prepDataLatih))])
  #================================================================

  logArr = []
  df = []
  idf = []
  TFIDF = True
  output.clear()
  #print("Prepare for Count TFIDF: " + str(len(bobotData)))

  for i in range(len(bobotData)): #--> Loop setiap Term
    print("Kata Ke-" + str(i+1) + " dari " + str(len(bobotData)), end=" : ")
    logValue = []
    dfValue = 0
    idfValue = 0
    for j in range(len(bobotData.iloc[i])): #--> Loop setiap Dokumen pada Term ke-i
      #bobotData.iloc[i][j] = 0
      raw = prepDataLatih.iloc[j][2].count(uniques[i])
      log = 0
      if uniques[i] in prepDataLatih.iloc[j][2]:
        log = 1 + math.log10(raw) #--> Menghitung LogTF apabila Term yang dicheck muncul pada list stemming prepDataLatih pada Dokumen j
        dfValue += 1      
      logValue.append(log)
      bobotData.iloc[i][j] = log #raw #--> Memasukan nilai LogTF pada Term ke-i yang sudah dicheck kemunculannya pada Dokumen j
      #idf.append(log)
    print(logValue)
    logArr.append(logValue)
    df.append(dfValue)
    idf.append(math.log10(len(bobotData.iloc[i])/dfValue))
    output.clear()

  if (TFIDF): #--> Menghitung TFIDF dan disimpan pada bobotData
    for i in range(len(bobotData)):
      for j in range(len(bobotData.iloc[i])):
        bobotData.iloc[i][j] = logArr[i][j] * idf[i]

  bobotData['df'] = df
  bobotData['idf'] = idf

  return bobotData

#**KLASIFIKASI**

##**PRIOR**

In [15]:
def Prior(check, classif):
  prior = []

  for i in check:
    prior.append((classif.count(i))/len(classif))

  print("  Prior Done. . .")
  return prior

##**LIKELIHOOD**

In [16]:
def Likelihood(bobotData, check, dfDataLatih, dfStem):
  multinomialModel = []
  #dokumen = []

  for index, value in enumerate(check):
    dokumen = []
    valueEachTerm = []
    valueEachDoc = []
    valueEachClassif = []

    for j in range(len(dfDataLatih)):
      if dfDataLatih.iloc[j][3] == value:
        dokumen.append(j)
  
    for j in dokumen:
      sumDoc = 0
      for k in range(len(bobotData)):
        sumDoc += bobotData.iloc[k][j]
      valueEachDoc.append(sumDoc)
    #print(valueEachDoc)

    for j in range(len(bobotData)):
      sumTerm = 0
      for k in dokumen:
        sumTerm += bobotData.iloc[j][k]
      valueEachTerm.append(sumTerm)
    #print(valueEachTerm)

    for j in range(len(bobotData)):
      valueEachClassif.append((valueEachTerm[j]+1)/(sum(valueEachDoc)+len(bobotData)))

    multinomialModel.append(valueEachClassif)
  
  likelihood = LikelihoodUji(dfStem, dfDataLatih, multinomialModel, check)
  
  print("  Likelihood Done. . .")
  return likelihood

In [27]:
def LikelihoodUji(dfStem, dfDataLatih, multinomialModel, check):
  likelihood = []

  key_diff = set(dfStem.index).difference(dfDataLatih.index)
  where_diff = dfStem.index.isin(key_diff)

  prepDataUji = dfStem[where_diff]

  for i in prepDataUji.index.values.tolist():
    likelihoodEachDoc = []
    likelihoodEachTerm = []
    termUji = prepDataUji.loc[i]['stemming']
  #print(termUji)

    #print("Query ke-" + str(i), end=": ")
    for index, value in enumerate(termUji):
      notEmpty = False
      for j in uniques:
        valueEachUji = []
        if value == j:
          for k in range(len(check)):
            #valueEachUji.append(multinomialModel[k][index])
            valueEachUji.append(multinomialModel[k][uniques.index(j)])
          notEmpty = True
          break
    #print(valueEachUji)
      if notEmpty:
        likelihoodEachTerm.append(valueEachUji)
    #print(likelihoodEachTerm)
      #print(likelihoodEachTerm)
    
    for j in range(len(check)):
      likelhoodEachClass = 1
      for index, value in enumerate(likelihoodEachTerm):
        #likelhoodEachClass *= value[j]
        likelhoodEachClass *= value[j]*1000
      likelihoodEachDoc.append(np.prod(likelhoodEachClass))
  
    likelihood.append(likelihoodEachDoc)

  return likelihood

##**POSTERIOR**

In [18]:
def Posterior(dfStem, dfDataLatih, prior, likelihood):
  posterior = []

  key_diff = set(dfStem.index).difference(dfDataLatih.index)
  where_diff = dfStem.index.isin(key_diff)

  prepDataUji = dfStem[where_diff]

  for i in range(len(prepDataUji)):
    posteriorEachDoc = []
  
    for j in range(len(likelihood[i])):
      posteriorEachDoc.append(prior[j] * likelihood[i][j])
  
    posterior.append(posteriorEachDoc)  

  print("  Posterior Done. . .")
  return posterior

#**MAIN PROGRAM**

In [19]:
def saveFile(dfDataUji, check, posterior, skenario):
  dataResult = []
  maxValue = 0
  for i in dfDataUji.index.values.tolist():
    result = []
    result.append("Q"+str(i+1))
    result.append(check[posterior[maxValue].index(max(posterior[maxValue]))])
    result.append(max(posterior[maxValue]))
    dataResult.append(result)

    maxValue += 1

  fileName = "Skenario-3."+str(skenario+1)+".csv"
  dfFile = pd.DataFrame(dataResult, columns=["Dokumen", "Kelas", "Value"])
  dfFile.to_csv(fileName, index=False)
  files.download(fileName)
#  return ""

In [34]:
# MELAKUKAN PERHITUNGAN PERSENTASE ANTARA DATA LATIH & DATA UJI
#persentase = [60, 70, 80, 90]
persentase = [90]
runTime = []
totalDataset = len(dfDataset)

for skenario, value in enumerate(persentase):
  start_time = time.time()                         
  drainData = round((totalDataset*value)/100)     

  print("============================================= SKENARIO - "+ str(skenario+1) + " =============================================")
  print("Jumlah Data Latih = " + str(drainData))
  print("Jumlah Data Uji   = " + str(totalDataset-drainData))
  
  print("\n> Memisahkan Data Latih & Data Uji")
  # MEMBUAT DATAFRAME BARU ANTARA POSITIF & NEGATIF
  dfPositif = dfDataset.loc[dfDataset['Kelas Berita'] == 'Positif']
  dfNegatif = dfDataset.loc[dfDataset['Kelas Berita'] == 'Negatif']

  dfDataLatih = getDataLatih(dfPositif, dfNegatif, drainData)
  dfDataUji = getDataUji(dfDataset, dfDataLatih)
  print("  Done. . .")

  print("\n> Melakukan Text Preprocessing")
  # MENYIAPKAN KALIMAT UNTUK PROSES TEXT PREPROCESSING
  dfDataLatih["Raw Text"] = dfDataLatih["Judul Berita"] + " " + dfDataLatih["Isi Berita"]
  dfDataUji["Raw Text"] = dfDataUji["Judul Berita"] + " " + dfDataUji["Isi Berita"]

  dfToken = Tokenizing(dfDataLatih, dfDataUji)
  dfFilter = Filtering(dfToken)
  dfStem = Stemming(dfFilter)
  
  # MENGHITUNG TF-IDF
  key_diff = set(dfStem.index).difference(dfDataUji.index)
  where_diff = dfStem.index.isin(key_diff)

  dfPrepDataLatih = dfStem[where_diff]
  uniques = TermUniques(dfPrepDataLatih)
  dfTFIDF = TermWeighting(dfPrepDataLatih, uniques)
  print("  Done. . .")

  print("\n> Melakukan Analisis Sentimen")
  # MENGAMBIL KLASIFIKASI SENTIMEN
  classif = dfDataLatih["Kelas Berita"]
  classif = classif.values.tolist()
  check = [] 
  for i in classif: 
    if i not in check: 
      check.append(i)

  # MELAKUKAN KLASIFIKASI NAIVE BAYES
  priorValue = Prior(check, classif)
  likelihoodValue = Likelihood(dfTFIDF, check, dfDataLatih, dfStem)
  posteriorValue = Posterior(dfStem, dfDataLatih, priorValue, likelihoodValue)
  print("  Done. . .")

  print("\n> Hasil Klasifikasi Masing-Masing Data Uji")
  # MENGAMBIL NILAI TERTINGGI (PELUANG TERTINGGI)
  posterior = posteriorValue
  maxValue = 0
  for i in dfDataUji.index.values.tolist():
    print("  Dokumen Uji " + str(i+1) + " Memiliki Posterior Terbesar: "  + str(max(posterior[maxValue])), end=" | Termasuk Kelas: " + check[posterior[maxValue].index(max(posterior[maxValue]))] + "\n")
    maxValue += 1

  runTime.append(round(time.time() - start_time, 2))
  print("\n--- %.2f seconds ---" % (time.time() - start_time))
  saveFile(dfDataUji, check, posterior, skenario)
  

  Done. . .

> Melakukan Analisis Sentimen
  Prior Done. . .
  Likelihood Done. . .
  Posterior Done. . .
  Done. . .

> Hasil Klasifikasi Masing-Masing Data Uji
  Dokumen Uji 90 Memiliki Posterior Terbesar: 5.260158393332624e-12 | Termasuk Kelas: Negatif
  Dokumen Uji 92 Memiliki Posterior Terbesar: 3.570965398927244e-05 | Termasuk Kelas: Positif
  Dokumen Uji 93 Memiliki Posterior Terbesar: 8.977132773476901e-08 | Termasuk Kelas: Negatif
  Dokumen Uji 94 Memiliki Posterior Terbesar: 1.9897561370765613e-10 | Termasuk Kelas: Positif
  Dokumen Uji 95 Memiliki Posterior Terbesar: 1.0370031779657513e-15 | Termasuk Kelas: Negatif
  Dokumen Uji 96 Memiliki Posterior Terbesar: 8.711263299716413e-13 | Termasuk Kelas: Positif
  Dokumen Uji 97 Memiliki Posterior Terbesar: 1.4659095693640226e-15 | Termasuk Kelas: Negatif
  Dokumen Uji 98 Memiliki Posterior Terbesar: 8.046126176323535e-09 | Termasuk Kelas: Positif
  Dokumen Uji 99 Memiliki Posterior Terbesar: 0.024332024862781968 | Termasuk Kelas

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
runTime

[241.7, 272.46, 305.58, 349.04]

In [ ]:
posteriorValue

[[8.112083706356534e-107, 1.0173410952115076e-112],
 [1.0174248159797738e-173, 9.851889959821559e-169]]

#DUMP

##DUMP TYPE

In [ ]:
uploaded = files.upload()

dfRAW = pd.read_excel(io.BytesIO(uploaded['Type.xlsx']))


Saving Type.xlsx to Type.xlsx


In [ ]:
listStem = [j for sub in dfRAW.values.tolist() for j in sub]

dfStem = pd.DataFrame(listStem, columns=["Type"])

#dfStem = pd.DataFrame(columns=["Type"])
#dfStem = dfStem.append({'Type': listStem}, ignore_index=True)

factory = StemmerFactory()
stemmer = factory.create_stemmer()

dfStem['Stemming'] = dfStem['Type'].apply(lambda x: stemmer.stem(x))
#dfStem['Stemming'] = dfStem['Type'].apply(lambda x: [stemmer.stem(y) for y in x])

#dfStem

In [ ]:
dfDisplay = pd.DataFrame(index=dfStem["Stemming"].values.tolist())
display(dfDisplay)

""
prabowo
muak
elite
jakarta
bajing
...
menang
masalah
angka
adik


In [ ]:
dfDisplay.to_excel("cobaType.xlsx", index=True)
files.download("cobaType.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##DUMP TYPE DOC TRAIN



In [ ]:
uploaded = files.upload()

dfRAW = pd.read_excel(io.BytesIO(uploaded['TypeDoc.xlsx']))


Saving TypeDoc.xlsx to TypeDoc.xlsx


In [ ]:
dfStem = pd.DataFrame(columns=dfRAW.columns.values.tolist())

factory = StemmerFactory()
stemmer = factory.create_stemmer()

for kolom in dfRAW.columns.values.tolist():
  dfStem[kolom] = dfRAW[kolom].apply(lambda x: stemmer.stem(x) if not pd.isnull(x) else "")

#df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')

dfStem.to_excel("cobaTypeDoc.xlsx", index=True)
files.download("cobaTypeDoc.xlsx")
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##TOKENIZING & FILTERING

In [ ]:
# MEMBUAT FUNCTION TOKENIZE UNTUK MEMISAHKAN SETIAP KATA PADA DATA 
def Tokenize(rddValue):
  punc_Num='â¢Â€Ã!"#$%&\'()*+,:;<=>?@[\\]^_`{|}~0123456789'
  for ch in punc_Num:                       #--> PERULANGAN UNTUK MENGECEK CHARACTER BERDASARKAN "punc_Num"
    rddValue = rddValue.replace(ch, "")     #--> MENGHAPUS CHARACTER 
  rddValue = rddValue.replace(".", " ")     #--> MENGGANTIKAN CHARACTER "." MENJADI WHITESPACE " "
  rddValue = rddValue.replace("-", " ")     #--> MENGGANTIKAN CHARACTER "-" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("/", " ")     #--> MENGGANTIKAN CHARACTER "/" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("‚", " ")     #--> MENGGANTIKAN CHARACTER "," MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\xa0", " ")  #--> MENGGANTIKAN CHARACTER "\xa0" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x80\x9c", " ")  #--> MENGGANTIKAN CHARACTER "\x80\x9c" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x80", " ")  #--> MENGGANTIKAN CHARACTER "\x80" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x93", " ")  #--> MENGGANTIKAN CHARACTER "\x93" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x94", " ")  #--> MENGGANTIKAN CHARACTER "\x94" MENJADI WHITESPACE " "
  rddValue = rddValue.replace("\x99", " ")  #--> MENGGANTIKAN CHARACTER "\x99" MENJADI WHITESPACE " "

  rddToken = rddValue.split()   #--> MEMISAHKAN SETIAP KATA
  
  return rddToken

In [ ]:
stopList = []
listBerita = ["cnn", "cnnindonesia", "cnnindonesiacom", "detik", "detikcom", "liputan", "liputancom", "inipasti", "inipasticom", "antaranews", "antaranewscom", "com"]

url = "http://static.hikaruyuuki.com/wp-content/uploads/stopword_list_tala.txt"
headers = {'Accept-Encoding': 'identity'}
r = requests.get(url, headers=headers); ind = 0
for line in r.text.splitlines():
    #print(line)
    stopList.append(line)
    ind += 1
    if len(r.text.splitlines()) == ind:
      for berita in listBerita:
        stopList.append(berita)

#preprocessing['filtering'] = preprocessing['tokenized'].apply(lambda x: [item for item in x if item.lower() not in stopList])

#preprocessing

In [ ]:
uploaded = files.upload()

dfRAW = pd.read_excel(io.BytesIO(uploaded['TokenDoc.xlsx']))


Saving TokenDoc.xlsx to TokenDoc.xlsx


In [ ]:
combine = []
for row in dfRAW.index.values.tolist():
  string = dfRAW.iloc[row][0] + " " + dfRAW.iloc[row][1]
  combine.append(string)

tokenize = []
for text in combine:
  tokenize.append(Tokenize(text))

df = pd.DataFrame([[tokenize[0]]])
for i, v in enumerate(tokenize):
  if i == len(tokenize)-1:
    break
  df = df.append([[tokenize[i+1]]])
  
display(df)

,0
0,"[Prabowo, Muak, dengan, Elite, di, Jakarta, Me..."
0,"[Jokowi, Pembuat, Kebohongan, Itu, Tidak, Miki..."
0,"[Jokowi, Saya, Tanya, PUBG, Dijawab, Pak, Prab..."
0,"[Pemantau, Pemilu, Asing, Indonesia, Seharusny..."
0,"[Tips, Jokowi, Jaga, Kebugaran, Tubuh, Selama,..."
0,"[BPN, Sebut, KPU, Pulihkan, Ribu, DPT, Bermasa..."
0,"[Debat, Kelima, TKN, Prabowo, Tak, Nyambung, T..."
0,"[Pesan, Politik, SBY, Jelang, Pemungutan, Suar..."


In [ ]:
df[1] = df[0].apply(lambda x: [item for item in x if item.lower() not in stopList])

display(df)

,0,1
0,"[Prabowo, Muak, dengan, Elite, di, Jakarta, Me...","[Prabowo, Muak, Elite, Jakarta, Bajingan, Baji..."
0,"[Jokowi, Pembuat, Kebohongan, Itu, Tidak, Miki...","[Jokowi, Pembuat, Kebohongan, Mikir, Karawang,..."
0,"[Jokowi, Saya, Tanya, PUBG, Dijawab, Pak, Prab...","[Jokowi, PUBG, Prabowo, Pertanian, Nggak, Nyam..."
0,"[Pemantau, Pemilu, Asing, Indonesia, Seharusny...","[Pemantau, Pemilu, Asing, Indonesia, Bangga, J..."
0,"[Tips, Jokowi, Jaga, Kebugaran, Tubuh, Selama,...","[Tips, Jokowi, Jaga, Kebugaran, Tubuh, Kampany..."
0,"[BPN, Sebut, KPU, Pulihkan, Ribu, DPT, Bermasa...","[BPN, KPU, Pulihkan, Ribu, DPT, Bermasalah, Ja..."
0,"[Debat, Kelima, TKN, Prabowo, Tak, Nyambung, T...","[Debat, TKN, Prabowo, Nyambung, TKN, Jokowi, M..."
0,"[Pesan, Politik, SBY, Jelang, Pemungutan, Suar...","[Pesan, Politik, SBY, Jelang, Pemungutan, Suar..."


In [ ]:
#teksList = dfStem["stemming"].values.tolist()
teksList = df[1].values.tolist()
print(teksList)

df1 = pd.DataFrame(teksList)
display(df1)

[['prabowo', 'muak', 'elite', 'jakarta', 'bajing', 'bajing', 'yogyakarta', 'calon', 'presiden', 'prabowo', 'subianto', 'buka', 'pidato', 'bangsa', 'stadion', 'kridosono', 'yogyakarta', 'kutip', 'pidato', 'soekarno', 'adil', 'belanda', 'bunyi', 'gera', 'ronta', 'lahir', 'hasut', 'kaum', 'intelektual', 'gera', 'lahir', 'alamiah', 'derita', 'rakyat', 'tahan', 'saudara', 'paham', 'negara', 'negara', 'sakit', 'ketidakadilan', 'parah', 'republik', 'republik', 'juang', 'soekarno', 'ahmad', 'dahlan', 'hasyim', 'asyari', 'prabowo', 'rap', 'api', 'senin', 'prabowo', 'pertiwi', 'perkosa', 'hak', 'rakyat', 'injak', 'injak', 'gelintir', 'elite', 'jakarta', 'rusak', 'negara', 'bicara', 'keras', 'bajing', 'bajing', 'prabowo', 'aku', 'muak', 'elite', 'jakarta', 'bumn', 'bangga', 'negeri', 'milik', 'rakyat', 'garuda', 'pertamina', 'pln', 'rampok', 'bukti', 'bukti', 'bpk', 'bpk', 'bas', 'ribu', 'temu', 'tindak', 'bukti', 'polisi', 'tentara', 'polisi', 'tentara', 'milik', 'rakyat', 'bela', 'antek', 'ante

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280
0,prabowo,muak,elite,jakarta,bajing,bajing,yogyakarta,calon,presiden,prabowo,subianto,buka,pidato,bangsa,stadion,kridosono,yogyakarta,kutip,pidato,soekarno,adil,belanda,bunyi,gera,ronta,lahir,hasut,kaum,intelektual,gera,lahir,alamiah,derita,rakyat,tahan,saudara,paham,negara,negara,sakit,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,jokowi,buat,bohong,mikir,karawang,capres,nomor,urut,joko,widodo,jokowi,dukung,karawang,tangkal,hoaks,fitnah,jelang,delapan,coblos,pilpres,jokowi,sebut,orang,bohong,mikir,cerita,kait,hoaks,turun,pki,jokowi,analogi,umur,pki,bubar,turut,logika,hoaks,masuk,logika,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,jokowi,pubg,prabowo,tani,nggak,nyambung,capres,tahana,joko,widodo,jokowi,singgung,capres,nomor,urut,prabowo,subianto,tani,tanya,esport,jokowi,nilai,prabowo,sinkron,tanya,kembang,esport,mobile,legend,cerita,pubg,dota,tani,nggak,sambung,ya,nggak,maaf,jokowi,panggung,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,pantau,milu,asing,indonesia,bangga,jakarta,indonesia,pantau,milu,asal,negara,pantau,asing,puji,selenggara,milu,april,kepala,media,komunikasi,duta,malaysia,indonesia,john,nickell,alam,keliling,tps,momen,fantastis,indonesia,momen,orang,indonesia,bangga,praktik,demokrasi,sukses,damai,john,...,national,democratic,institute,ndi,world,bank,group,world,bank,ngo,negeri,kumpul,milu,demokrasi,perludem,indonesia,corruption,watch,icw,komite,independen,pantau,milu,kipp,indonesian,parliamentary,center,ipc,forum,indonesia,transparansi,anggar,fitra,pusat,pilih,akses,sandang,cacat,ppua,penca
4,tips,jokowi,jaga,bugar,tubuh,kampanye,mamuju,calon,presiden,jawat,joko,widodo,jokowi,aku,milik,tips,jaga,bugar,tubuh,lewat,kampanye,buka,pilpres,padat,inti,makan,minum,jokowi,kampanye,buka,lapang,ahmad,kirang,mamuju,sulawesi,barat,sulbar,kamis,turut,makan,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,bpn,kpu,pulih,ribu,dpt,masalah,jakarta,indonesia,badan,menang,nasional,bpn,prabowo,subianto,sandiaga,uno,sebut,komisi,pilih,kpu,hasil,baik,data,ribu,juta,daftar,pilih,dpt,masalah,apresiasi,kerja,kpu,sisir,data,wajar,hasil,kpu,data,pilih,baik,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,debat,tkn,prabowo,nyambung,tkn,jokowi,maruf,kritik,capres,prabowo,subianto,debat,pilpres,prabowo,nyambung,paslon,prabowo,milik,konsistensi,pikir,kuasa,ekonomi,negara,nyambung,paham,cawapres,sandi,tim,tugas,khusus,tkn,nusyirwan,soejono,wartawan,sabtu,nusyirwan,ketidaknyambungan,prabowo,sesi,debat,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,pesan,politik,sby,jelang,mungut,suara,milu,jakarta,indonesia,presiden,enam,republik,indonesia,susilo,bambang,yudhoyono,sby,pesan,politik,masyarakat,indonesia,kader,partai,jelang,mungut,suara,milu,ketua,partai,demokrat,harap,milu,sarana,bawa,indonesia,negara,sby,harap,bangsa,indonesi,...,None,None

In [ ]:
df1 = df1.T

In [ ]:
df1.to_excel("cobaa.xlsx", index=False)
files.download("cobaa.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def TermWeighting(prepDataLatih, uniques):
  bobotData = pd.DataFrame(index=[teks for teks in uniques], columns=[("D" + str(i+1)) for i in range(len(prepDataLatih))])
  #================================================================

  logArr = []
  df = []
  idf = []
  TFIDF = True
  output.clear()
  #print("Prepare for Count TFIDF: " + str(len(bobotData)))

  for i in range(len(bobotData)): #--> Loop setiap Term
    print("Kata Ke-" + str(i+1) + " dari " + str(len(bobotData)), end=" : ")
    logValue = []
    dfValue = 0
    idfValue = 0
    for j in range(len(bobotData.iloc[i])): #--> Loop setiap Dokumen pada Term ke-i
      #bobotData.iloc[i][j] = 0
      raw = 0
      log = 0
      for k in range(len(prepDataLatih.iloc[j][2])): #--> Loop setiap list stemming prepDataLatih pada Dokumen j
        if (uniques[i] == prepDataLatih.iloc[j][2][k]): #--> Check apabila Term ke-i sama dengan indeks k pada list stemming prepDataLatih pada Dokumen j
          #bobotData.iloc[i][j] += 1
          raw += 1
      if (raw > 0):
        log = 1 + math.log10(raw) #--> Menghitung LogTF apabila Term yang dicheck muncul pada list stemming prepDataLatih pada Dokumen j
        dfValue += 1      
      logValue.append(log)
      bobotData.iloc[i][j] = log #raw #--> Memasukan nilai LogTF pada Term ke-i yang sudah dicheck kemunculannya pada Dokumen j
      #idf.append(log)
    print(logValue)
    logArr.append(logValue)
    df.append(dfValue)
    idf.append(math.log10(len(bobotData.iloc[i])/dfValue))
    output.clear()

  #rawTerm.append(idf) 
  #total = []
  #for i in range(len(bobotData.columns)):
  #  total.append(0)
  #for i in range(len(bobotData)):
  #  for j in range(len(bobotData.columns)):
  #    total[j] += bobotData.iloc[i][j]

  if (TFIDF): #--> Menghitung TFIDF dan disimpan pada bobotData
    for i in range(len(bobotData)):
      for j in range(len(bobotData.iloc[i])):
        bobotData.iloc[i][j] = logArr[i][j] * idf[i]

  #bobotData.loc['TOTAL'] = total
  #bobotData['log'] = logArr

  bobotData['df'] = df
  bobotData['idf'] = idf

  return bobotData